# Advanced Features: AI_COMPLETE and Helper Functions

This notebook demonstrates:
- **AI_COMPLETE**: Text generation with LLMs
- **AI_COUNT_TOKENS**: Token counting for optimization
- **PROMPT**: Dynamic prompt building
- Helper functions for production use


In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from snowflake.snowpark import Session
from IPython.display import display, Markdown, HTML

# Try to get active session (for Snowflake Notebooks)
# Otherwise, connect using ~/.snowflake/connections.toml
try:
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
    print("✅ Connected using Snowflake Notebooks session")
except:
    import toml
    from pathlib import Path
    toml_path = Path.home() / ".snowflake" / "connections.toml"
    if toml_path.exists():
        connections = toml.load(toml_path)
        connection_name = "default"
        if connection_name in connections:
            conn_params = connections[connection_name]
            session = Session.builder.configs(conn_params).create()
            print(f"✅ Connected using connection profile: {connection_name}")
        else:
            raise Exception(f"❌ Connection '{connection_name}' not found")
    else:
        raise Exception(f"❌ Connection file not found at {toml_path}")

session.sql("USE DATABASE AISQL_DB").collect()
session.sql("USE SCHEMA AISQL_SCHEMA").collect()
session.sql("USE WAREHOUSE AISQL_WH").collect()


✅ Connected using connection profile: vinodshiv


[Row(status='Statement executed successfully.')]

## 1. AI_COMPLETE: Text Generation

Generate AI responses for customer support tickets


In [13]:
# Generate responses with AI_COMPLETE
# Note: Cortex Guard may not be available in all regions/accounts
# Using basic AI_COMPLETE without options for compatibility
sql = """
SELECT 
    ticket_id,
    SUBSTR(content, 1, 100) as customer_message,
    AI_COMPLETE('llama3.1-8b', 
        'Generate a brief professional response to: ' || SUBSTR(content, 1, 200)) as ai_response
FROM emails
LIMIT 10
"""

df_complete = session.sql(sql).to_pandas()
display(Markdown("### AI_COMPLETE Results"))
display(Markdown("**Note:** Cortex Guard (`guard_enable` option) may not be available in all regions. Using basic AI_COMPLETE for compatibility."))
display(df_complete)


### AI_COMPLETE Results

**Note:** Cortex Guard (`guard_enable` option) may not be available in all regions. Using basic AI_COMPLETE for compatibility.

,TICKET_ID,CUSTOMER_MESSAGE,AI_RESPONSE
0,1830,I'd appreciate a refund for the unused Saturda...,"""Here is a potential response:\n\nDear [Custom..."
1,1462,"Also, quick heads up - there seems to be a gli...","""Here is a brief professional response:\n\n\""T..."
2,177,"While I have you, I wanted to share some feedb...","""Here's a potential response:\n\n\""Thank you s..."
3,632,"Also, the new paperless ticket system is gener...","""Here's a possible response:\n\n\""I appreciate..."
4,1813,"Also, I purchased tickets for the Summer Elect...","""Here is a brief professional response:\n\nDea..."
5,1320,"Additionally, I wanted to provide some feedbac...","""Here's a potential response:\n\n\""I appreciat..."
6,466,I'm also wondering if you could clarify your r...,"""Here is a potential response:\n\n\""I apprecia..."
7,603,One suggestion - it would be amazing if you co...,"""Here's a brief professional response:\n\n\""Th..."
8,1033,I urgently need assistance with transferring m...,"""Here is a brief professional response:\n\n\""H..."
9,681,"Also, I purchased VIP passes for the summer mu...","""Here is a brief professional response:\n\n\""D..."


## 2. AI_COUNT_TOKENS: Token Management

Count tokens for cost optimization


In [7]:
# Count tokens for cost estimation
# AI_COUNT_TOKENS syntax: AI_COUNT_TOKENS('function_name', 'model_name', input_text)
# Note: claude-3-7-sonnet is NOT supported by AI_COUNT_TOKENS, using llama3.3-70b instead
sql = """
SELECT 
    ticket_id,
    SUBSTR(content, 1, 100) as content_preview,
    LENGTH(content) as char_count,
    AI_COUNT_TOKENS('ai_complete', 'llama3.3-70b', content) as token_count,
    ROUND((AI_COUNT_TOKENS('ai_complete', 'llama3.3-70b', content) / 1000.0) * 0.003, 6) as estimated_cost_usd
FROM emails
ORDER BY token_count DESC
LIMIT 20
"""

df_tokens = session.sql(sql).to_pandas()
display(Markdown("### Token Analysis"))
display(df_tokens)

# Token distribution
display(Markdown("### Token Count Distribution"))
fig = px.histogram(df_tokens, 
                   x='TOKEN_COUNT', 
                   nbins=20,
                   title='Distribution of Token Counts',
                   labels={'TOKEN_COUNT': 'Token Count', 'count': 'Count'})
fig.update_layout(height=400)
fig.show()

# Cost summary
total_tokens = df_tokens['TOKEN_COUNT'].sum()
total_cost = df_tokens['ESTIMATED_COST_USD'].sum()
display(Markdown("### Cost Summary"))
display(Markdown(f"**Total Tokens:** {total_tokens:,}"))
display(Markdown(f"**Estimated Cost:** ${total_cost:.4f}"))


### Token Analysis

,TICKET_ID,CONTENT_PREVIEW,CHAR_COUNT,TOKEN_COUNT,ESTIMATED_COST_USD
0,1208,I recently attended the indie folk festival at...,479,109,0.000327
1,1520,"While I have you, I wanted to mention that I'v...",468,102,0.000306
2,519,I've been trying to transfer my tickets for ne...,412,97,0.000291
3,268,I've been trying to get this sorted out for da...,402,96,0.000288
4,208,I attended last weekend's indie folk festival ...,434,93,0.000279
5,1454,"While I've got your attention, I also wanted ...",416,90,0.000270
6,1320,"Additionally, I wanted to provide some feedbac...",437,86,0.000258
7,1146,I'm reaching out about my VIP ticket purchase ...,340,84,0.000252
8,177,"While I have you, I wanted to share some feedb...",370,83,0.000249
9,1843,I purchased two VIP tickets for the upcoming e...,319,82,0.000246


### Token Count Distribution

### Cost Summary

**Total Tokens:** 1,685

**Estimated Cost:** $0.0051

## 3. PROMPT Function: Dynamic Prompts

Build dynamic prompts with parameters


In [8]:
# Use PROMPT for dynamic prompt building
sql = """
SELECT 
    ticket_id,
    user_id,
    SUBSTR(content, 1, 100) as content_preview,
    AI_COMPLETE('llama3.1-8b',
        PROMPT('User {0} submitted ticket {1}. Please provide a helpful response to: {2}',
               user_id, ticket_id, content)) as personalized_response
FROM emails
LIMIT 5
"""

df_prompt = session.sql(sql).to_pandas()
display(Markdown("### Dynamic Prompt Results"))
display(df_prompt)


### Dynamic Prompt Results

,TICKET_ID,USER_ID,CONTENT_PREVIEW,PERSONALIZED_RESPONSE
0,1830,14,I'd appreciate a refund for the unused Saturda...,"Here's a potential response:\n\nDear User 14,\..."
1,1462,29,"Also, quick heads up - there seems to be a gli...","Here's a potential response:\n\n""Thank you for..."
2,177,166,"While I have you, I wanted to share some feedb...",Thank you so much for taking the time to share...
3,632,57,"Also, the new paperless ticket system is gener...","Here's a potential response:\n\n""Hi User 57, t..."
4,1813,91,"Also, I purchased tickets for the Summer Elect...","Here's a potential response:\n\n""Hello User 91..."


## Summary

This notebook covered advanced Cortex AISQL features:
- **AI_COMPLETE** for text generation with LLMs
- **AI_COUNT_TOKENS** for cost optimization
- **PROMPT** for dynamic prompt construction
- Helper functions for production use

These functions help build production-ready AI applications with proper cost management and dynamic prompt handling.

**Note:** Cortex Guard (`guard_enable` option) may not be available in all regions/accounts. Check the Snowflake documentation for regional availability.
